# Classidication Crime Category

In [0]:
from pyspark.sql.functions import *
# Import StringIndexer class
from pyspark.ml.feature import StringIndexer
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import Bucketizer
# Import the necessary class
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
# Import class for creating a pipeline
from pyspark.ml import Pipeline
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
#import SparkSession
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('SF_CRIME').getOrCreate()

In [0]:
df_sf = spark.read.format("csv").load("./Police_Department_Incident_Reports__Historical_2003_to_May_2018_1.csv",sep=',',header=True,inferSchema=True ,nullValue='NONE')
df_sf = df_sf.withColumnRenamed('Incident Code','IncidentCode')

In [0]:
df_sf = df_sf.withColumn("Hour", hour(col("Time")))\
              .withColumn("Minute", minute(col("Time")))\
              .withColumn("Date", to_date(col("Date"), "MM/dd/yyyy"))\
              .withColumn("Year", year(col("Date")))\
              .withColumn("Month", month(col("Date")))\
              .withColumn("Day", dayofmonth(col("Date")))\
              .withColumn("Week", weekofyear(col("Date")))\
              .withColumn("X", df_sf.X.cast("double"))\
              .withColumn("Y", df_sf.Y.cast("double"))

In [0]:
# dropna
df_sf = df_sf.filter('PdDistrict is not NULL')

In [0]:
df_sf.dtypes

Out[26]: [('PdId', 'double'),
 ('IncidntNum', 'int'),
 ('IncidentCode', 'int'),
 ('Category', 'string'),
 ('Descript', 'string'),
 ('DayOfWeek', 'string'),
 ('Date', 'date'),
 ('Time', 'string'),
 ('PdDistrict', 'string'),
 ('Resolution', 'string'),
 ('Address', 'string'),
 ('X', 'double'),
 ('Y', 'double'),
 ('location', 'string'),
 ('Hour', 'int'),
 ('Minute', 'int'),
 ('Year', 'int'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Week', 'int')]

In [0]:
if df_sf.is_cached:
  df_sf.unpersist()
df_sf = df_sf.cache()
display(df_sf)

PdId,IncidntNum,IncidentCode,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,Hour,Minute,Year,Month,Day,Week
1.30001E13,130001186,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,2013-01-01,10:12,SOUTHERN,PSYCHOPATHIC CASE,300 Block of MISSION ST,-122.3962112,37.79076136,POINT (-122.396211154038 37.790761361966),10,12,2013,1,1,1
1.30004E13,130003518,28160,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,2013-01-01,17:30,RICHMOND,null,1400 Block of CLEMENT ST,-122.4746799,37.78242784,POINT (-122.474679889558 37.782427844456706),17,30,2013,1,1,1
1.3026E13,130260445,75025,NON-CRIMINAL,SEARCH WARRANT SERVICE,Tuesday,2013-01-01,0:01,BAYVIEW,"ARREST, BOOKED",23RD ST / WISCONSIN ST,-122.3986956,37.7547456,POINT (-122.39869558968901 37.7547456011189),0,1,2013,1,1,1
1.3E13,130000081,4134,ASSAULT,BATTERY,Tuesday,2013-01-01,0:39,SOUTHERN,null,MARKET ST / 1ST ST,-122.3991523,37.79101661,POINT (-122.39915226019902 37.7910166130682),0,39,2013,1,1,1
1.30003E13,130003007,65015,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,2013-01-01,23:10,CENTRAL,"ARREST, BOOKED",LARKIN ST / BUSH ST,-122.4186583,37.78890984,POINT (-122.418658294829 37.7889098399421),23,10,2013,1,1,1
1.30002E13,130001910,4134,ASSAULT,BATTERY,Tuesday,2013-01-01,15:23,SOUTHERN,EXCEPTIONAL CLEARANCE,200 Block of 6TH ST,-122.4063464,37.77916742,POINT (-122.40634642563201 37.7791674218963),15,23,2013,1,1,1
1.30511E13,130511157,5013,BURGLARY,"BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY",Tuesday,2013-01-01,0:01,TARAVAL,null,0 Block of THOMASMORE WY,-122.4729374,37.71225228,POINT (-122.47293744666898 37.7122522752262),0,1,2013,1,1,1
1.30002E13,130002093,4013,ASSAULT,AGGRAVATED ASSAULT WITH A DEADLY WEAPON,Tuesday,2013-01-01,16:33,SOUTHERN,"ARREST, BOOKED",9TH ST / MISSION ST,-122.4147143,37.77623104,POINT (-122.414714295579 37.7762310404758),16,33,2013,1,1,1
1.30003E13,130002952,3472,ROBBERY,ATTEMPTED ROBBERY WITH A KNIFE,Tuesday,2013-01-01,23:10,TENDERLOIN,null,OFARRELL ST / JONES ST,-122.4129705,37.78578838,POINT (-122.412970537591 37.7857883766888),23,10,2013,1,1,1
1.30652E13,130651822,19057,ASSAULT,THREATS AGAINST LIFE,Tuesday,2013-01-01,8:00,INGLESIDE,COMPLAINANT REFUSES TO PROSECUTE,600 Block of SAN JOSE AV,-122.4218052,37.74593903,POINT (-122.42180518927502 37.7459390289798),8,0,2013,1,1,1


In [0]:
indexer_df_sf = StringIndexer(inputCol='Category',outputCol='Category_idx')
# Assign index values to strings
indexer_df_sf = indexer_df_sf.fit(df_sf)
# Create column with index values
df_sf = indexer_df_sf.transform(df_sf)
display(df_sf)
df_sf.select('Category','Category_idx').distinct().show()

PdId,IncidntNum,IncidentCode,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,Hour,Minute,Year,Month,Day,Week,Category_idx
1.30001E13,130001186,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,2013-01-01,10:12,SOUTHERN,PSYCHOPATHIC CASE,300 Block of MISSION ST,-122.3962112,37.79076136,POINT (-122.396211154038 37.790761361966),10,12,2013,1,1,1,2.0
1.30004E13,130003518,28160,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,2013-01-01,17:30,RICHMOND,null,1400 Block of CLEMENT ST,-122.4746799,37.78242784,POINT (-122.474679889558 37.782427844456706),17,30,2013,1,1,1,4.0
1.3026E13,130260445,75025,NON-CRIMINAL,SEARCH WARRANT SERVICE,Tuesday,2013-01-01,0:01,BAYVIEW,"ARREST, BOOKED",23RD ST / WISCONSIN ST,-122.3986956,37.7547456,POINT (-122.39869558968901 37.7547456011189),0,1,2013,1,1,1,2.0
1.3E13,130000081,4134,ASSAULT,BATTERY,Tuesday,2013-01-01,0:39,SOUTHERN,null,MARKET ST / 1ST ST,-122.3991523,37.79101661,POINT (-122.39915226019902 37.7910166130682),0,39,2013,1,1,1,3.0
1.30003E13,130003007,65015,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,2013-01-01,23:10,CENTRAL,"ARREST, BOOKED",LARKIN ST / BUSH ST,-122.4186583,37.78890984,POINT (-122.418658294829 37.7889098399421),23,10,2013,1,1,1,1.0
1.30002E13,130001910,4134,ASSAULT,BATTERY,Tuesday,2013-01-01,15:23,SOUTHERN,EXCEPTIONAL CLEARANCE,200 Block of 6TH ST,-122.4063464,37.77916742,POINT (-122.40634642563201 37.7791674218963),15,23,2013,1,1,1,3.0
1.30511E13,130511157,5013,BURGLARY,"BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY",Tuesday,2013-01-01,0:01,TARAVAL,null,0 Block of THOMASMORE WY,-122.4729374,37.71225228,POINT (-122.47293744666898 37.7122522752262),0,1,2013,1,1,1,7.0
1.30002E13,130002093,4013,ASSAULT,AGGRAVATED ASSAULT WITH A DEADLY WEAPON,Tuesday,2013-01-01,16:33,SOUTHERN,"ARREST, BOOKED",9TH ST / MISSION ST,-122.4147143,37.77623104,POINT (-122.414714295579 37.7762310404758),16,33,2013,1,1,1,3.0
1.30003E13,130002952,3472,ROBBERY,ATTEMPTED ROBBERY WITH A KNIFE,Tuesday,2013-01-01,23:10,TENDERLOIN,null,OFARRELL ST / JONES ST,-122.4129705,37.78578838,POINT (-122.412970537591 37.7857883766888),23,10,2013,1,1,1,11.0
1.30652E13,130651822,19057,ASSAULT,THREATS AGAINST LIFE,Tuesday,2013-01-01,8:00,INGLESIDE,COMPLAINANT REFUSES TO PROSECUTE,600 Block of SAN JOSE AV,-122.4218052,37.74593903,POINT (-122.42180518927502 37.7459390289798),8,0,2013,1,1,1,3.0


+--------------------+------------+
 Category|Category_idx|
+--------------------+------------+
 RECOVERED VEHICLE| 19.0|
 SECONDARY CODES| 14.0|
SEX OFFENSES, FOR...| 17.0|
 WARRANTS| 6.0|
 EMBEZZLEMENT| 26.0|
 ASSAULT| 3.0|
 NON-CRIMINAL| 2.0|
 BURGLARY| 7.0|
 MISSING PERSON| 10.0|
 VEHICLE THEFT| 5.0|
 ARSON| 25.0|
 LARCENY/THEFT| 0.0|
DRIVING UNDER THE...| 23.0|
 ROBBERY| 11.0|
 OTHER OFFENSES| 1.0|
 WEAPON LAWS| 13.0|
 SUSPICIOUS OCC| 8.0|
 VANDALISM| 4.0|
 STOLEN PROPERTY| 16.0|
 FRAUD| 12.0|
+--------------------+------------+
only showing top 20 rows

In [0]:
# Split into training and testing sets in a 80:20 ratio
df_sf_train, df_sf_test = df_sf.randomSplit([.8,.2], seed=17)

In [0]:
# Categorical columns

# Create an indexer
indexer = StringIndexer(inputCols=['DayOfWeek','PdDistrict'], outputCols=['DayOfWeek_idx','PdDistrict_idx'])

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['DayOfWeek_idx','PdDistrict_idx'], outputCols=['DayOfWeek_dummy','PdDistrict_dummy'])

# Create an assembler object
assembler = VectorAssembler(inputCols=['DayOfWeek_dummy','PdDistrict_dummy'], outputCol='features')

logistic = LogisticRegression(labelCol='Category_idx')

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler,logistic])

# Train the pipeline on the training data
pipeline = pipeline.fit(df_sf_train)

# Make predictions on the testing data
predictions = pipeline.transform(df_sf_test)

In [0]:
# 计算测试误差
evaluator = MulticlassClassificationEvaluator(
    labelCol="Category_idx", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-186073013367678> in <module> 
 2 evaluator = MulticlassClassificationEvaluator(
 3 labelCol="Category_idx", predictionCol="prediction", metricName="accuracy")
 ----> 4 accuracy = evaluator . evaluate ( predictions ) 
 5 print ( "Test Error = %g " % ( 1.0 - accuracy ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 70 return self . copy ( params ) . _evaluate ( dataset ) 
 71 else : 
 ---> 72 return self . _evaluate ( dataset ) 
 73 else : 
 74 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 100 """
 101 self . _transfer_params_to_java ( ) 
 --> 102 return self . _java_obj . evaluate ( dataset . _jdf ) 
 103 
 104 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 IllegalArgumentException : prediction does not exist. Available: PdId, IncidntNum, IncidentCode, Category, Descript, DayOfWeek, Date, Time, PdDistrict, Resolution, Address, X, Y, location, Hour, Minute, Year, Month, Day, Week, Category_idx, Week_idx, Day_idx, PdDistrict_idx, DayOfWeek_idx, Hour_idx, Year_idx, Month_idx, PdDistrict_dummy, DayOfWeek_dummy, Day_dummy, Hour_dummy, Week_dummy, Month_dummy, Year_dummy, features

In [0]:
display(predictions)

PdId,IncidntNum,IncidentCode,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,Hour,Minute,Year,Month,Day,Week,Category_idx,DayOfWeek_idx,PdDistrict_idx,DayOfWeek_dummy,PdDistrict_dummy,features,rawPrediction,probability,prediction
1.30016E11,1300164,75000,MISSING PERSON,FOUND PERSON,Thursday,2014-02-13,9:00,RICHMOND,LOCATED,700 Block of 48TH AV,-122.5089545,37.77422811,POINT (-122.50895453984 37.7742281136725),9,0,2014,2,13,7,10.0,3.0,9.0,"Map(vectorType -> sparse, length -> 6, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 9, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 15, indices -> List(3), values -> List(1.0))","Map(vectorType -> dense, length -> 38, values -> List(4.553078273342561, 3.5859435154836925, 3.795203790432931, 2.6874769849877493, 2.7600568617445274, 2.776951019153811, 1.9916791947590045, 2.7209259160934156, 2.5569952634751303, 1.332435816750145, 1.868944809862764, 1.3549961913534405, 1.8999999558992329, 0.2131645239952408, 1.0133929446735572, 0.42627060423012636, 0.4588590028276317, 0.33737658801878045, 0.433711508219625, -0.04543663478347182, -0.3666546672967381, 0.17065825554449, -0.7663882083291129, 0.1345398039406489, -1.0497765082534574, -0.7999393264536723, -1.2120630580497762, -1.313093107664503, -2.2564661520387816, -2.7418724151974345, -2.4745613807666698, -2.1405020139186366, -2.9064262175633098, -2.4291968812802063, -2.662715935157706, -3.5262284286527117, -4.7607943929948275, -5.620545496387496))","Map(vectorType -> dense, length -> 38, values -> List(0.32199217729353014, 0.12241201086805353, 0.1509053423237263, 0.049845387411816516, 0.053597683000070756, 0.054510862674721425, 0.02485672084577495, 0.05154086014053149, 0.04374792754415952, 0.012856953514768711, 0.02198573398185637, 0.013150307851901175, 0.022679216529007874, 0.004198022167110843, 0.009345004500116669, 0.00519511926540594, 0.005367208712842385, 0.0047532356033710265, 0.0052339198256731116, 0.0032414227150390613, 0.0023508907772697503, 0.004023319198947317, 0.0015762691691468425, 0.0038805961227892424, 0.0011872888962373425, 0.0015242609220948923, 0.001009430105615659, 9.124297982796008E-4, 3.552202802301289E-4, 2.186192957619016E-4, 2.8561446473173856E-4, 3.988966029578946E-4, 1.8544865276080708E-4, 2.98869605268477E-4, 2.3662777138768303E-4, 9.978080479717539E-5, 2.9032324982073E-5, 1.228843726368413E-5))",0.0
1.3084E12,13083956,61030,NON-CRIMINAL,"DEATH REPORT, CAUSE UNKNOWN",Friday,2013-05-10,12:50,TENDERLOIN,null,100 Block of TURK ST,-122.4116151,37.78316141,POINT (-122.411615075408 37.7831614109395),12,50,2013,5,10,19,2.0,0.0,7.0,"Map(vectorType -> sparse, length -> 6, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 9, indices -> List(7), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(0, 13), values -> List(1.0, 1.0))","Map(vectorType -> dense, length -> 38, values -> List(3.845590903497494, 3.523395839837581, 3.6249956501756615, 3.380896858169797, 2.1147761290021307, 1.1403818355025546, 3.0423184731020365, 1.689710507515288, 2.456031845274886, 3.265264703536051, 1.4921004656027597, 2.123976798484331, 1.5088882754806812, 1.3211500338195186, 0.7580208839484344, 1.097064209547205, 0.28386517691815494, 0.41282741090645625, 0.24260845232459877, -0.04876295521584609, 0.37168071929611024, -0.311994161280291, 0.04170710002091127, -0.9633832240381839, -0.28722360972310174, -0.9614604352074629, -0.9893441942933503, -0.8975321223929604, -1.9832913270934842, -2.195022377169816, -2.2053108567255246, -2.6838187171123975, -2.170011792140527, -4.07003549106456, -4.669201589101088, -2.829521944535906, -4.642084284844738, -5.829253190023409))","Map(vectorType -> dense, length -> 38, values -> List(0.17948191510437234, 0.13004484957451987, 0.14395189741239742, 0.11277338989154961, 0.03179343038732265, 0.011999504812960709, 0.08038294452833405, 0.020784220216675937, 0.04472422911435576, 0.1004

In [0]:
# Create an indexer
indexer = StringIndexer(inputCols=['DayOfWeek','PdDistrict','Hour','Month','Year','Day','Week'], outputCols=['DayOfWeek_idx','PdDistrict_idx','Hour_idx','Month_idx','Year_idx','Day_idx','Week_idx'])

In [0]:
# split_Hour=[0,1, 7, 13, 23]

# bucketizer = Bucketizer(splitsArray=[split_Hour],inputCols=['Hour'],outputCols=['Hour_bin'])

In [0]:

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['DayOfWeek_idx','PdDistrict_idx','Hour_idx','Month_idx','Year_idx','Day_idx','Week_idx'], outputCols=['DayOfWeek_dummy','PdDistrict_dummy','Hour_dummy','Month_dummy','Year_dummy','Day_dummy','Week_dummy'])

In [0]:
# Create an assembler object
assembler = VectorAssembler(inputCols=['DayOfWeek_dummy','PdDistrict_dummy','Hour_dummy','Month_dummy','Year_dummy','Day_dummy','Week_dummy'], outputCol='features')


In [0]:
forest = RandomForestClassifier(numTrees=5)

In [0]:


# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler])

# Train the pipeline on the training data
pipeline = pipeline.fit(df_sf_train)

# Make predictions on the testing data
predictions = pipeline.transform(df_sf_test)

In [0]:
display(df_sf.groupby('Hour').count().sort('Hour'))

Hour,count
0,39470
1,23394
2,18988
3,13488
4,9705
5,8859
6,12273
7,19428
8,27830
9,31115


In [0]:
display(df_sf.groupby('Day').count().sort('Day'))

Day,count
1,30033
2,25664
3,25811
4,25895
5,26099
6,25894
7,26069
8,25553
9,26014
10,26510


In [0]:
display(df_sf.groupby('Month').count().sort('Month'))

Month,count
1,75860
2,68377
3,75054
4,71826
5,67159
6,61201
7,63333
8,65027
9,63118
10,65790


In [0]:
display(df_sf.groupby('Year','Month').count().sort('Year','Month'))

Year,Month,count
2013,1,12461
2013,2,11436
2013,3,12617
2013,4,12145
2013,5,12556
2013,6,12153
2013,7,12810
2013,8,13383
2013,9,13482
2013,10,13586


In [0]:
split_Hour=[0,1, 7, 13, 23]

from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splitsArray=[split_Hour],inputCols=['Hour'],outputCols=['Hour_bin'])
